In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid", palette="pastel")

In [2]:
rl_kpis = pd.read_csv("/root/projects/ci_anomaly_detection/data/turkcell/final_preprocessed_df_train.csv")

In [3]:
rl_kpis['datetime']=pd.to_datetime(rl_kpis['datetime'])

In [4]:
df_labels = rl_kpis[["datetime", "site_id", "mlid"]]
df_labels.head()

,datetime,site_id,mlid
0,2018-12-31,RL_;ABDV,A0BE
1,2018-12-31,RL_;ABDV,A0BI
2,2018-12-31,RL_;ABDV,A5AB
3,2018-12-31,RL_;ABDV,A8CQ
4,2018-12-31,RL_;ABDV,A8DQ


In [5]:
prediction_interval = 5

for i in range(prediction_interval):
  df_labels[f"T+{i+1}"] = df_labels["datetime"] + pd.DateOffset(days=i+1)
df_labels.head()

/tmp/ipykernel_11418/3134115453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[f"T+{i+1}"] = df_labels["datetime"] + pd.DateOffset(days=i+1)
/tmp/ipykernel_11418/3134115453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labels[f"T+{i+1}"] = df_labels["datetime"] + pd.DateOffset(days=i+1)
/tmp/ipykernel_11418/3134115453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,datetime,site_id,mlid,T+1,T+2,T+3,T+4,T+5
0,2018-12-31,RL_;ABDV,A0BE,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05
1,2018-12-31,RL_;ABDV,A0BI,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05
2,2018-12-31,RL_;ABDV,A5AB,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05
3,2018-12-31,RL_;ABDV,A8CQ,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05
4,2018-12-31,RL_;ABDV,A8DQ,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05


In [6]:
rl_kpis_view = rl_kpis[["datetime", "site_id", "mlid", "rlf"]]
for i in range(prediction_interval):
  target_day_column_name = f"T+{i+1}"

  df_labels = df_labels.merge(rl_kpis_view, 
                  how = "left", 
                  left_on = ("site_id", "mlid", target_day_column_name),
                  right_on = ("site_id", "mlid", "datetime"),
                  suffixes = ("", "_y")
  )
  df_labels.rename(columns={"rlf": f"{target_day_column_name}_rlf"}, inplace=True)
df_labels.drop(columns=["datetime_y"], inplace=True)
df_labels.head()

,datetime,site_id,mlid,T+1,T+2,T+3,T+4,T+5,T+1_rlf,T+2_rlf,T+3_rlf,T+4_rlf,T+5_rlf
0,2018-12-31,RL_;ABDV,A0BE,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,0.0,0.0,0.0,0.0,0.0
1,2018-12-31,RL_;ABDV,A0BI,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,0.0,0.0,0.0,0.0,0.0
2,2018-12-31,RL_;ABDV,A5AB,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,0.0,0.0,0.0,0.0,0.0
3,2018-12-31,RL_;ABDV,A8CQ,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,0.0,0.0,0.0,0.0,0.0
4,2018-12-31,RL_;ABDV,A8DQ,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,0.0,0.0,0.0,0.0,0.0


In [7]:
# 1 day predict is equal to T+1 rlf
df_labels["1-day-predict"] = df_labels["T+1_rlf"]

# Interval predict (5-day predict) is based on T+1, T+2, T+3, T+4 and T+5
following_days_rlf_columns = [f"T+{i+1}_rlf" for i in range(prediction_interval)]

df_labels["5-day-predict"] = df_labels[following_days_rlf_columns].any(axis=1)
df_labels = df_labels[["datetime", "site_id", "mlid", "1-day-predict", "5-day-predict"]]

print(f"df_labels.shape: {df_labels.shape}")
print(f"df_labels 1-day rlf sum: {df_labels['1-day-predict'].sum()}")
print(f"df_labels 5-day rlf sum: {df_labels['5-day-predict'].sum()}")
df_labels.head()

df_labels.shape: (1992986, 5)
df_labels 1-day rlf sum: 1204.0
df_labels 5-day rlf sum: 5159


,datetime,site_id,mlid,1-day-predict,5-day-predict
0,2018-12-31,RL_;ABDV,A0BE,0.0,False
1,2018-12-31,RL_;ABDV,A0BI,0.0,False
2,2018-12-31,RL_;ABDV,A5AB,0.0,False
3,2018-12-31,RL_;ABDV,A8CQ,0.0,False
4,2018-12-31,RL_;ABDV,A8DQ,0.0,False


In [8]:
# Now join labels with rl-kpis
rl_kpis_with_labels = rl_kpis.merge(df_labels, 
                                    how="left", 
                                    on=["datetime", "site_id", "mlid"])
rl_kpis_with_labels.head()

,type,datetime,tip,mlid,mw_connection_no,site_id,card_type,adaptive_modulation,freq_band,severaly_error_second,...,humidity_mean_3,precipitation_mean_3,precipitation_coeff_mean_3,bbe_3,error_second_3,severaly_error_second_3,error_second_3.1,unavail_second_3,1-day-predict,5-day-predict
0,ENK,2018-12-31,FAR,A0BE,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False
1,ENK,2018-12-31,FAR,A0BI,"1,349,988",RL_;ABDV,cardtype1,Enable,f3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False
2,ENK,2018-12-31,FAR,A5AB,"1,344,018",RL_;ABDV,cardtype4,Enable,f3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False
3,NEC,2018-12-31,FAR,A8CQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False
4,NEC,2018-12-31,FAR,A8DQ,"1,351,204",RL_;ABDV,cardtype5,Enable,f2,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,False


## Model Training

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

In [10]:
list(rl_kpis.select_dtypes(include=['object']).columns)

['type',
 'tip',
 'mlid',
 'mw_connection_no',
 'site_id',
 'card_type',
 'adaptive_modulation',
 'freq_band',
 'modulation']

In [11]:
categorical_features =  ["card_type", "freq_band",]
numerical_features = list(set(rl_kpis.columns) - set(categorical_features) - set(['type','tip','mlid','mw_connection_no','site_id','adaptive_modulation','modulation','datetime']))

features = categorical_features + numerical_features

In [12]:
# Simple undersampling
np.random.seed(1234)

cond_rlf = rl_kpis_with_labels["5-day-predict"]
rlf_count = cond_rlf.sum()
print("rlf count: ", rlf_count)

# Get sample index from non rlf columns with 1:3 ratio
sampled_non_rlf_indicies = np.random.choice(rl_kpis_with_labels[~cond_rlf].index, size=rlf_count * 2)
rlf_indicies = np.array(rl_kpis_with_labels[cond_rlf].index)

sampled_data_indicies = list(sampled_non_rlf_indicies) + list(rlf_indicies)
sampled_data = rl_kpis_with_labels.loc[sampled_data_indicies]
sampled_data.shape

rlf count:  5159


(15477, 89)

In [13]:
df_train, df_test = train_test_split(sampled_data, test_size=0.2)
print(f"df_train.shape: {df_train.shape} | df_test.shape: {df_test.shape}")

df_train.shape: (12381, 89) | df_test.shape: (3096, 89)


In [14]:
# Convert categorical columns to one hot vector
# Merge them with numerical columns
# Return X data, column names, and encoder for future usage

from sklearn.preprocessing import OneHotEncoder


def preprocessing(df, numerical_columns=[], categorical_columns=[], one_hot_encoder=None):
    df = df.dropna()

    # Проверка наличия столбцов в DataFrame
    for col in numerical_columns + categorical_columns:
        if col not in df.columns:
            raise ValueError(f"Столбец '{col}' отсутствует в DataFrame!")

    if one_hot_encoder is None:
        one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        one_hot_encoder.fit(df[categorical_columns])
        print("*" * 50)
        print("Названия признаков:", one_hot_encoder.get_feature_names_out(categorical_columns))
        print("*" * 50)
        print("Категории:", one_hot_encoder.categories_)

    # Преобразуем числовые столбцы в массив NumPy
    arr_numerical = df[numerical_columns].to_numpy()

    try:
        arr_categorical = one_hot_encoder.transform(df[categorical_columns])
    except ValueError as e:
        raise ValueError(f"Ошибка при трансформации категориальных данных: {e}")

    if arr_numerical.shape[0] != arr_categorical.shape[0]:
        raise ValueError("Количество строк в числовых и категориальных данных не совпадает!")

    arr_x = np.concatenate((arr_numerical, arr_categorical), axis=1)

    feature_names = numerical_columns + one_hot_encoder.get_feature_names_out(categorical_columns).tolist()

    return df.copy(), arr_x, feature_names, one_hot_encoder


df_train_dropped, train_x, feature_names, one_hot_encoder = preprocessing(
    df_train,
    numerical_columns=numerical_features,
    categorical_columns=categorical_features
)

Создаётся новый OneHotEncoder...
OneHotEncoder создан и обучен:
**************************************************
Названия признаков: ['card_type_cardtype1' 'card_type_cardtype10' 'card_type_cardtype11'
 'card_type_cardtype2' 'card_type_cardtype4' 'card_type_cardtype5'
 'card_type_cardtype6' 'freq_band_f0' 'freq_band_f1' 'freq_band_f2'
 'freq_band_f3' 'freq_band_f4' 'freq_band_f5']
**************************************************
Категории: [array(['cardtype1', 'cardtype10', 'cardtype11', 'cardtype2', 'cardtype4',
       'cardtype5', 'cardtype6'], dtype=object), array(['f0', 'f1', 'f2', 'f3', 'f4', 'f5'], dtype=object)]


In [15]:
clf_1_day_pred = RandomForestClassifier(class_weight='balanced')

clf_5_day_pred = RandomForestClassifier(class_weight='balanced')

In [16]:
# train_x is prepared at preprocessing step
train_y_1_day_pred = df_train_dropped["1-day-predict"].astype('int').to_numpy()
train_y_5_day_pred = df_train_dropped["5-day-predict"].astype('int').to_numpy()

In [17]:
_ = clf_1_day_pred.fit(train_x, train_y_1_day_pred)
_= clf_5_day_pred.fit(train_x, train_y_5_day_pred)

In [18]:

# Preprocess test data
df_test_dropepd, test_x, _, _ = preprocessing(df_test, numerical_columns=numerical_features, 
                             categorical_columns=categorical_features, 
                             one_hot_encoder=one_hot_encoder)
test_y_1_day_pred = df_test_dropepd["1-day-predict"].astype('int').to_numpy()
test_y_5_day_pred = df_test_dropepd["5-day-predict"].astype('int').to_numpy()

pred_1_day = clf_1_day_pred.predict(test_x)
pred_5_day = clf_5_day_pred.predict(test_x)
pred_1_day.sum(), pred_5_day.sum()
     

(166, 832)

## Results

In [19]:
precision, recall, fscore, _ = precision_recall_fscore_support(pred_1_day, 
                                                               test_y_1_day_pred, 
                                                               average="binary", # 
                                                               labels=[0, 1], # labels
                                                               beta=1) # f1 score

print("*********** SCORE for 1-DAY predict")
print(f"precision : {precision:.4f}")
print(f"recall    : {recall:.4f}")
print(f"f-score   : {fscore:.4f}")


precision, recall, fscore, _ = precision_recall_fscore_support(pred_5_day,        # y'
                                                               test_y_5_day_pred, # y
                                                               average="binary",  # focus only True class
                                                               labels=[0, 1],     # labels
                                                               beta=1)            # f1 score
print()
print("*********** SCORE for 5-DAY predict ***********")
print(f"precision : {precision:.4f}")
print(f"recall    : {recall:.4f}")
print(f"f-score   : {fscore:.4f}")

*********** SCORE for 1-DAY predict
precision : 0.5592
recall    : 0.8253
f-score   : 0.6667

*********** SCORE for 5-DAY predict ***********
precision : 0.6979
recall    : 0.8329
f-score   : 0.7595
